# Train

In [ ]:
!cd ../.. ; python src/models/Pix2pixHD/train.py \
--continue_train --no_lpips_loss --l1_pre \
--lambda_l1 1 --lambda_feat 3  --no_flip \
--niter 100 --niter_decay 100 \
--checkpoints_dir temp/checkpoint/Pix2pixHD/ \
--name conditional_binary_spectral_l1_pre \
--dataroot data/xBD_merged \
--dataset_mode physics_aligned_bin \
--no_instance --label_nc 0 --input_nc 4 --batchSize 8 --gpu_ids 0,1,2,3,4,5,6,7

In [ ]:
#cp pretrained model into finetuning directory
#Make a copy of the fully trained model to be finetuned on xBD_floods
!cd ../../temp/checkpoint/Pix2pixHD/; mkdir conditional_binary_spectral_l1_pre_full_reconstruct ;  cp conditional_binary_spectral_l1_pre/latest* conditional_binary_spectral_l1_pre_full_reconstruct/ ; cp conditional_binary_spectral_l1_pre/iter* conditional_binary_spectral_l1_pre_full_reconstruct/

In [ ]:
!cd ../.. ; python src/models/Pix2pixHD/train.py --continue_train --no_lpips_loss --l1_pre --lambda_l1 3 --lambda_feat 3  --no_flip --niter 205 --niter_decay 0 --checkpoints_dir temp/checkpoint/Pix2pixHD/ --name conditional_binary_spectral_l1_pre_full_reconstruct --dataroot data/xBD_flood_augmented_noelastic --dataset_mode physics_aligned_bin --no_instance --label_nc 0 --input_nc 4 --batchSize 8 --gpu_ids 0,1,2,3,4,5,6,7

In [ ]:
!cd ../.. ; python src/models/Pix2pixHD/train.py --continue_train --save_epoch_freq 2 --l1_pre_init_prob 0.9 --no_lpips_loss --l1_pre --lambda_l1 2 --lambda_feat 3  --no_flip --niter 210 --niter_decay 0 --checkpoints_dir temp/checkpoint/Pix2pixHD/ --name conditional_binary_spectral_l1_pre_full_reconstruct --dataroot data/xBD_2events --dataset_mode physics_aligned_bin --no_instance --label_nc 0 --input_nc 4 --batchSize 8 --gpu_ids 0,1,2,3,4,5,6,7

In [ ]:
!cd ../.. ; python src/models/Pix2pixHD/train.py --continue_train --save_epoch_freq 2 --l1_pre_init_prob 0.75 --no_lpips_loss --l1_pre --lambda_l1 2 --lambda_feat 3  --no_flip --niter 230 --niter_decay 0 --checkpoints_dir temp/checkpoint/Pix2pixHD/ --name conditional_binary_spectral_l1_pre_full_reconstruct --dataroot data/xBD_2events --dataset_mode physics_aligned_bin --no_instance --label_nc 0 --input_nc 4 --batchSize 8 --gpu_ids 0,1,2,3,4,5,6,7

In [ ]:
!cd ../.. ; python src/models/Pix2pixHD/train.py --continue_train --save_epoch_freq 20 --l1_pre_init_prob 0.5 --no_lpips_loss --l1_pre --lambda_l1 10 --lambda_feat 3  --no_flip --niter 600 --niter_decay 400 --checkpoints_dir temp/checkpoint/Pix2pixHD/ --name conditional_binary_spectral_l1_pre_full_reconstruct --dataroot data/xBD_2events --dataset_mode physics_aligned_bin --no_instance --label_nc 0 --input_nc 4 --batchSize 8 --gpu_ids 0,1,2,3,4,5,6,7

In [7]:
#Generate predictions over the test set. 
!cd ../.. ; python src/models/Pix2pixHD/test.py  --checkpoints_dir temp/checkpoint/Pix2pixHD/ --name conditional_binary_spectral_l1_pre --phase test --how_many 2000  --dataroot data/xBD --dataset_mode physics_aligned_bin --no_instance --label_nc 0 --batchSize 8 --input_nc 4 --gpu_ids 0,1,2,3,4,5,6,7 --results_dir temp/Pix2pixHD/


------------ Options -------------
aspect_ratio: 1.0
augmentation_aggressive: False
batchSize: 8
checkpoints_dir: temp/checkpoint/Pix2pixHD/
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: data/xBD
dataset_mode: physics_aligned_bin
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0, 1, 2, 3, 4, 5, 6, 7]
how_many: 2000
input_nc: 4
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 8
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: conditional_binary_spectral_l1_pre
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
normD: spectral
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: scale_width
results_dir: temp/Pix2pixHD/
serial_batches: False
slosh_inference: False
tf_log: False
us

### We move things around at the moment in ./Temp/
The model output is placed into ./temp/ .We use this directory to work, then we carefully selects what goes into ./results/.
./temp/ does not get uploaded into our bucket, only ./results/ do!

In [8]:
# Move synthesized results to a folder to be used by the segmentation network
!cd ../../temp/Pix2pixHD/conditional_binary_spectral_l1_pre/test_latest/ ; mkdir to_segment; cp -R images/*synthesized_image.jpg to_segment

In [9]:
#Create Segmentation Masks
#Generate masks for all of the data generated by pix2pixHD_baseline
! cd ../.. ; python src/models/Pix2pix-CycleGAN/test.py --phase to_segment --dataroot temp/Pix2pixHD/conditional_binary_spectral_l1_pre/test_latest/ --direction AtoB --results_dir ./temp/Pix2pixHD/conditional_binary_spectral_l1_pre_masks --dataset_mode single --model pix2pix --checkpoints_dir ./pretrained/Pix2pix-CycleGAN/flood_segmentation/scratch_1024_plus/ --name . --num_test 775 --no_flip --gpu_ids 0 --max_dataset_size 2000 --batch_size 1  --load_size 1024 --crop_size 1024


----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./pretrained/Pix2pix-CycleGAN/flood_segmentation/scratch_1024_plus/	[default: ./checkpoints]
                crop_size: 1024                          	[default: 256]
                 dataroot: temp/Pix2pixHD/conditional_binary_spectral_l1_pre/test_latest/	[default: None]
             dataset_mode: single                        	[default: aligned]
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
  

### Now we copy our generated images into the right ./Results/. directory

In [11]:
! cd ../.. ; mkdir results/Pix2pixHD/conditional_binary_spectral_l1_pre/ ; cp -R temp/Pix2pixHD/conditional_binary_spectral_l1_pre/test_latest/images/* results/Pix2pixHD/conditional_binary_spectral_l1_pre/ ; cp -R temp/Pix2pixHD/conditional_binary_spectral_l1_pre_masks/to_segment_latest/images/*image_fake_B.png results/Pix2pixHD/conditional_binary_spectral_l1_pre/

mkdir: cannot create directory ‘results/Pix2pixHD/conditional_binary_spectral_l1_pre/’: File exists
